In [3]:
import pandas as pd

from pyrdf2vec import RDF2VecTransformer
from pyrdf2vec.embedders import Word2Vec
from pyrdf2vec.graphs import KG
from pyrdf2vec.walkers import RandomWalker

from pprint import pprint

In [29]:
data = pd.read_csv("taxonomy_ranks/rank_0.csv", sep=",",)
entities = [entity[1][0] for entity in data[data.columns[1:]].iterrows()]
print(entities)

['http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C20181', 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C1908', 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C17828', 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C22188', 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C14250', 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C97325', 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C12218', 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C43431', 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C3910', 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C28428', 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C20189', 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C16612', 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C20047', 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C7057', 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C26548', 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C12913', 'http://ncicb.nci.nih.gov/

KeyError: 0

In [31]:
# Read a CSV file containing the entities we want to classify.
data = pd.read_csv("taxonomy_ranks/rank_0.csv", sep=",",)
entities = [entity[1][0] for entity in data[data.columns[1:]].iterrows()]
entities = entities[:5]
print(entities)
# [
#    "http://dbpedia.org/resource/Belgium",
#    "http://dbpedia.org/resource/France",
#    "http://dbpedia.org/resource/Germany",
# ]

# Define our knowledge graph (here: DBPedia SPARQL endpoint).
knowledge_graph = KG(
    "./data/tax_and_subset_NCIT.owl",
)
# Create our transformer, setting the embedding & walking strategy.
transformer = RDF2VecTransformer(
    Word2Vec(epochs=10),
    walkers=[RandomWalker(4, 10, with_reverse=False, n_jobs=2)],
    # verbose=1
)
# Get our embeddings.
embeddings, literals = transformer.fit_transform(knowledge_graph, entities)
pprint(embeddings)
# [
#     array([ 1.5737595e-04,  1.1333118e-03, -2.9838676e-04,  ..., -5.3064007e-04,
#             4.3192197e-04,  1.4529384e-03], dtype=float32),
#     array([-5.9027621e-04,  6.1689125e-04, -1.1987977e-03,  ...,  1.1066757e-03,
#            -1.0603866e-05,  6.6087965e-04], dtype=float32),
#     array([ 7.9996325e-04,  7.2907173e-04, -1.9482171e-04,  ...,  5.6251377e-04,
#             4.1435464e-04,  1.4478950e-04], dtype=float32)
# ]

pprint(literals)

['http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C20181', 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C1908', 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C17828', 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C22188', 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C14250']
[array([ 9.7702928e-03,  8.1651136e-03,  1.2809718e-03,  5.0975787e-03,
        1.4081288e-03, -6.4551616e-03, -1.4280510e-03,  6.4491653e-03,
       -4.6173059e-03, -3.9930656e-03,  4.9244044e-03,  2.7130984e-03,
       -1.8479753e-03, -2.8769434e-03,  6.0107317e-03, -5.7167388e-03,
       -3.2367026e-03, -6.4878250e-03, -4.2346325e-03, -8.5809948e-03,
       -4.4697891e-03, -8.5112294e-03,  1.4037776e-03, -8.6181965e-03,
       -9.9166557e-03, -8.2016252e-03, -6.7726658e-03,  6.6805850e-03,
        3.7845564e-03,  3.5616636e-04, -2.9579818e-03, -7.4283206e-03,
        5.3341867e-04,  4.9989222e-04,  1.9561886e-04,  8.5259555e-04,
        7.8633073e-04, -6.8160298e-05, -8.0070542e-03, 

In [32]:
len(embeddings[0])

100